# Bolig_Portal Webscraping

- It is webscraping of Bolig Portal website which is one of the most famous website to find the rental residence in all over the Denmark.
- Webscarping is performed only for the Copenhagen residences.
- BeautifulSoup is used for the webscraping
## Dataframe
    - Dataframe is consist of Apartments, Address, Rent, Rooms, Area(m2), URL(include all urls of the apartments)
    - Longitude and Latitude is added by using geopy.geocoders with import ArcGIS
    - Distance is measured in km from Copenhagen Central Station to Apartment address with the help of geopy great_circle
    - Copenhagen map is printed with display of 6 rooms apartment by using folium library

In [305]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from itertools import chain
from geopy.geocoders import ArcGIS
from geopy.distance import geodesic
from geopy.distance import great_circle
import folium

# URL
- The main url which is scraped

In [6]:
main_url="https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/"
soup= BeautifulSoup(requests.get(main_url).content, "lxml")

# Key Elements of First Page Scraped  

In [24]:
Apartments=[i.text for i in soup.find_all('div', class_="css-xr43j5-Text-Text")]
Address=[i.text for i in soup.find_all('div',class_='css-1wrf1k9-Text-Text')]
Rent=[i.text for i in soup.find_all("span", class_="css-sdrc0-Text-Text")]

# All URLS with key Elements
- It included all urls of all pages
- Key elements which are Apartments, Address and Rent scraped from all the pages.

In [ ]:
offst_inc=18

In [7]:
all_pages = "https://www.boligportal.dk/en/rental-properties/k%C3%B8benhavn/?offset="

In [10]:
%%time
all_urls=[]
Apartments=[]
Address=[]
Rent=[]
page = main_url
offst_inc=0
for i in range(80):  
    soup= BeautifulSoup(requests.get(page).content, "lxml")
    urls = soup.find_all('div', class_="css-boo8iq-Box-Box")
    Aprt= [i.text for i in soup.find_all('div', class_="css-xr43j5-Text-Text")]
    Add=[i.text for i in soup.find_all('div',class_='css-1wrf1k9-Text-Text')]
    Rnt=[i.text for i in soup.find_all("span", class_="css-sdrc0-Text-Text")]
    Apartments.append(Aprt)
    Address.append(Add)
    Rent.append(Rnt)   
    for i in range(len(urls)):
        all_urls.append("https://www.boligportal.dk"+urls[i].a['href'])

    offst_inc=offst_inc+18
    page= main_url + "?offset="+str(offst_inc)

Wall time: 1min 12s


## Key Elements Converted to List
    - Key elements were in nested list which are changed into list

In [12]:
Apartment = list(chain(*Apartments))
Rent= list(chain(*Rent))
Address= list(chain(*Address))

In [13]:
print(len(Apartment))
print(len(Rent))
print(len(Address))

1920
1920
1920


# Dataframe Created

In [57]:
df_bolig=pd.DataFrame({"Apartment": Apartment, "Rent": Rent, "Address": Address, "URL": all_urls})

In [15]:
df_bolig

,Apartment,Rent,Address,URL
0,2 rm. · Apartment · 60 m²,8.700 kr.,"København, Vigerslevvej",https://www.boligportal.dk/en/rental-apartment...
1,3 rm. · Apartment · 102 m²,16.500 kr.,"København, Hans Hedtofts Gade",https://www.boligportal.dk/en/rental-apartment...
2,2 rm. · Apartment · 60 m²,8.429 kr.,"København, Romsdalsgade",https://www.boligportal.dk/en/rental-apartment...
3,3 rm. · Apartment · 88 m²,12.000 kr.,"København, Guldborgvej",https://www.boligportal.dk/en/rental-apartment...
4,1 rm. · Room · 9 m²,3.500 kr.,"København, Charlotte Muncks Vej",https://www.boligportal.dk/en/rental-rooms/k%C...
...,...,...,...,...
1915,4 rm. · Apartment · 111 m²,18.215 kr.,"Hellerup, C V E Knuths Vej",https://www.boligportal.dk/lejligheder/helleru...
1916,3 rm. · Apartment · 96 m²,13.700 kr.,"Brønshøj, Præstegårds Allé",https://www.boligportal.dk/lejligheder/br%C3%B...
1917,4 rm. · Apartment · 149 m²,34.000 kr.,"Hellerup, Tuborg Bakkedrag",https://www.boligportal.dk/lejligheder/helleru...
1918,4 rm. · Apartment · 115 m²,15.875 kr.,"Søborg, Søborg Hovegade",https://www.boligportal.dk/lejligheder/s%C3%B8...


## Dataframe Info

In [17]:
df_bolig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Apartment  1920 non-null   object
 1   Rent       1920 non-null   object
 2   Address    1920 non-null   object
 3   URL        1920 non-null   object
dtypes: object(4)
memory usage: 60.1+ KB


# Columns Added & Modified
- Rent columns is modified only into numbers
- Rooms and Area(m2) column added with the help of Regular Expression

In [59]:
df_bolig['Rent'] = df_bolig['Rent'].str.extract(r'(\d+.\d+)', expand = False)

In [61]:
df_bolig['Rooms'] = df_bolig['Apartment'].str.extract(r"([0-9]+) .*", expand = False)

In [62]:
df_bolig["Area(m²)"] = df_bolig['Apartment'].str.extract(r"(\d+)(?!.*\d)", expand = False)

In [63]:
df_bolig.head(3)

,Apartment,Rent,Address,URL,Rooms,Area(m²)
0,2 rm. · Apartment · 60 m²,8.700,"København, Vigerslevvej",https://www.boligportal.dk/en/rental-apartment...,2,60
1,3 rm. · Apartment · 102 m²,16.500,"København, Hans Hedtofts Gade",https://www.boligportal.dk/en/rental-apartment...,3,102
2,2 rm. · Apartment · 60 m²,8.429,"København, Romsdalsgade",https://www.boligportal.dk/en/rental-apartment...,2,60


## Datatype is changed to Numeric

In [95]:
df_bolig[["Rent", "Rooms", "Area(m²)"]] = df_bolig[["Rent", "Rooms", "Area(m²)"]].apply(pd.to_numeric)

In [96]:
df_bolig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451 entries, 0 to 1450
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Apartment  1451 non-null   object 
 1   Rent       1451 non-null   float64
 2   Address    1451 non-null   object 
 3   URL        1451 non-null   object 
 4   Rooms      1451 non-null   int64  
 5   Area(m²)   1451 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 68.1+ KB


# Duplicated Apartment Deleted

- There are promoted apartments which were on every page because these were highly advertised
- Found and deleted those apartment to make dataframe unique

In [70]:
df_bolig.duplicated().sum()

469

In [66]:
dup=df_bolig[df_bolig.duplicated()]

In [68]:
dup.head(10)

,Apartment,Rent,Address,URL,Rooms,Area(m²)
42,3 rm. · Apartment · 100 m²,13.995,"Rødovre, Nørregårdsbakken",https://www.boligportal.dk/lejligheder/r%C3%B8...,3,100
43,4 rm. · Apartment · 111 m²,18.215,"Hellerup, C V E Knuths Vej",https://www.boligportal.dk/lejligheder/helleru...,4,111
44,3 rm. · Apartment · 96 m²,13.700,"Brønshøj, Præstegårds Allé",https://www.boligportal.dk/lejligheder/br%C3%B...,3,96
45,4 rm. · Apartment · 149 m²,34.000,"Hellerup, Tuborg Bakkedrag",https://www.boligportal.dk/lejligheder/helleru...,4,149
46,4 rm. · Apartment · 115 m²,15.875,"Søborg, Søborg Hovegade",https://www.boligportal.dk/lejligheder/s%C3%B8...,4,115
47,4 rm. · Apartment · 138 m²,19.296,"Hellerup, C V E Knuths Vej",https://www.boligportal.dk/lejligheder/helleru...,4,138
66,3 rm. · Apartment · 100 m²,13.995,"Rødovre, Nørregårdsbakken",https://www.boligportal.dk/lejligheder/r%C3%B8...,3,100
67,4 rm. · Apartment · 111 m²,18.215,"Hellerup, C V E Knuths Vej",https://www.boligportal.dk/lejligheder/helleru...,4,111
68,3 rm. · Apartment · 96 m²,13.700,"Brønshøj, Præstegårds Allé",https://www.boligportal.dk/lejligheder/br%C3%B...,3,96
69,4 rm. · Apartment · 149 m²,34.000,"Hellerup, Tuborg Bakkedrag",https://www.boligportal.dk/lejligheder/helleru...,4,149


## Duplicated Apartments Displayed

In [71]:
df_bolig.drop_duplicates(inplace= True)

In [80]:
df_bolig[37:47]

,Apartment,Rent,Address,URL,Rooms,Area(m²)
37,4 rm. · Apartment · 135 m²,17.106,"København, Nørregade",https://www.boligportal.dk/en/rental-apartment...,4,135
38,3 rm. · Apartment · 86 m²,13.600,"København, A.D. Jørgensens Vej",https://www.boligportal.dk/en/rental-apartment...,3,86
39,2 rm. · Apartment · 42 m²,10.750,"København, Kapselgade",https://www.boligportal.dk/en/rental-apartment...,2,42
40,4 rm. · Apartment · 115 m²,14.700,"København, Ørestads Boulevard",https://www.boligportal.dk/en/rental-apartment...,4,115
41,4 rm. · Apartment · 102 m²,15.495,"København, Teglholmsgade",https://www.boligportal.dk/en/rental-apartment...,4,102
48,2 rm. · Apartment · 82 m²,12.995,"København, Teglholmsgade",https://www.boligportal.dk/en/rental-apartment...,2,82
49,5 rm. · Apartment · 127 m²,25.000,"København, Vardegade",https://www.boligportal.dk/en/rental-apartment...,5,127
50,1 rm. · Apartment · 37 m²,8.300,"København, Bavnehøj Allé",https://www.boligportal.dk/en/rental-apartment...,1,37
51,2 rm. · Apartment · 68 m²,13.500,"København, Amerika Plads",https://www.boligportal.dk/en/rental-apartment...,2,68
52,3 rm. · Apartment · 79 m²,14.200,"København, Utterslevvej",https://www.boligportal.dk/en/rental-apartment...,3,79


In [81]:
df_bolig.reset_index(drop = True, inplace= True)

In [82]:
df_bolig[37:47]

,Apartment,Rent,Address,URL,Rooms,Area(m²)
37,4 rm. · Apartment · 135 m²,17.106,"København, Nørregade",https://www.boligportal.dk/en/rental-apartment...,4,135
38,3 rm. · Apartment · 86 m²,13.600,"København, A.D. Jørgensens Vej",https://www.boligportal.dk/en/rental-apartment...,3,86
39,2 rm. · Apartment · 42 m²,10.750,"København, Kapselgade",https://www.boligportal.dk/en/rental-apartment...,2,42
40,4 rm. · Apartment · 115 m²,14.700,"København, Ørestads Boulevard",https://www.boligportal.dk/en/rental-apartment...,4,115
41,4 rm. · Apartment · 102 m²,15.495,"København, Teglholmsgade",https://www.boligportal.dk/en/rental-apartment...,4,102
42,2 rm. · Apartment · 82 m²,12.995,"København, Teglholmsgade",https://www.boligportal.dk/en/rental-apartment...,2,82
43,5 rm. · Apartment · 127 m²,25.000,"København, Vardegade",https://www.boligportal.dk/en/rental-apartment...,5,127
44,1 rm. · Apartment · 37 m²,8.300,"København, Bavnehøj Allé",https://www.boligportal.dk/en/rental-apartment...,1,37
45,2 rm. · Apartment · 68 m²,13.500,"København, Amerika Plads",https://www.boligportal.dk/en/rental-apartment...,2,68
46,3 rm. · Apartment · 79 m²,14.200,"København, Utterslevvej",https://www.boligportal.dk/en/rental-apartment...,3,79


# CSV file of Dataframe Created 

In [84]:
df_bolig.to_csv('Bolig_Portal.csv')

# Extract Room1 Apartments

In [99]:
Room1=df_bolig.loc[df_bolig['Rooms'] == 1]

In [100]:
Room1

,Apartment,Rent,Address,URL,Rooms,Area(m²)
4,1 rm. · Room · 9 m²,3.500,"København, Charlotte Muncks Vej",https://www.boligportal.dk/en/rental-rooms/k%C...,1,9
8,1 rm. · Apartment · 32 m²,6.950,"København, Lindgreens Allé",https://www.boligportal.dk/en/rental-apartment...,1,32
15,1 rm. · Room · 12 m²,5.253,"København, Robert Jacobsens Vej",https://www.boligportal.dk/en/rental-rooms/k%C...,1,12
44,1 rm. · Apartment · 37 m²,8.300,"København, Bavnehøj Allé",https://www.boligportal.dk/en/rental-apartment...,1,37
49,1 rm. · Room · 14 m²,6.900,"København, Ved Stranden",https://www.boligportal.dk/en/rental-rooms/k%C...,1,14
...,...,...,...,...,...,...
1442,1 rm. · Room · 24 m²,5.999,"København, Carl Th. Dreyers Vej",https://www.boligportal.dk/en/rental-rooms/k%C...,1,24
1443,1 rm. · Room · 8 m²,6.500,"København, Dieselvej",https://www.boligportal.dk/en/rental-rooms/k%C...,1,8
1446,1 rm. · Room · 14 m²,4.500,"København, Nørrebrogade",https://www.boligportal.dk/en/rental-rooms/k%C...,1,14
1448,1 rm. · Room · 10 m²,5.765,"København, Pladehals Allé",https://www.boligportal.dk/en/rental-rooms/k%C...,1,10


# Longitude and Langitude

- Found longitude and latitude to measure the distance from the Copenhagen Central Station

In [258]:
y=df_bolig["Address"]

In [192]:
def addr_long(x):
    from geopy.geocoders import ArcGIS
    lat_ = []
    long_ = []
    points = []
    for i in x:
        loc = ArcGIS()
        addr = loc.geocode(i, timeout=None)
        if addr is None:
            lat_.append(None)
            long_.append(None)
        else:
            longitude = addr.longitude
            latitude = addr.latitude
            points.append((latitude, longitude))
    return points

## Copenhagen Central Station Latitude, Longitude

In [120]:
addr_long(["København, Bernstoffgade 1"])

[(55.67439305996968, 12.564873126715405)]

In [232]:
abc=[(55.67439305996968, 12.564873126715405)]*1451

In [233]:
len(abc)

1451

## Calculated the Latitude, Longitude of all Apartments

In [ ]:
Add_Lat_Lon = [] 
for i in y[0:len(y)]:
    x = addr_long([i])
    Add_Lat_Lon.append(x)

In [136]:
Add_Lat_Lon = list(chain(*Add_Lat_Lon))

## New Column Added for Latitude, Longitude

In [236]:
df_bolig["Add_Lat_Lon"] = Add_Lat_Lon

In [ ]:
df_bolig["CPH_Lat_Lon"] = abc

In [237]:
df_bolig.head(3)

,Apartment,Rent,Address,URL,Rooms,Area(m²),CPH_Lat_Lon,Add_Lat_Lon
0,2 rm. · Apartment · 60 m²,8.700,"København, Vigerslevvej",https://www.boligportal.dk/en/rental-apartment...,2,60,"(55.67439305996968, 12.564873126715405)","(55.657086753224576, 12.485280377686337)"
1,3 rm. · Apartment · 102 m²,16.500,"København, Hans Hedtofts Gade",https://www.boligportal.dk/en/rental-apartment...,3,102,"(55.67439305996968, 12.564873126715405)","(55.66296135575674, 12.573964013115852)"
2,2 rm. · Apartment · 60 m²,8.429,"København, Romsdalsgade",https://www.boligportal.dk/en/rental-apartment...,2,60,"(55.67439305996968, 12.564873126715405)","(55.6595067245476, 12.602015356442074)"


In [259]:
df_bolig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451 entries, 0 to 1450
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Apartment               1451 non-null   object 
 1   Rent                    1451 non-null   float64
 2   Address                 1451 non-null   object 
 3   URL                     1451 non-null   object 
 4   Rooms                   1451 non-null   int64  
 5   Area(m²)                1451 non-null   int64  
 6   Distance_Central_St_km  1451 non-null   float64
 7   CPH_Lat_Lon             1451 non-null   object 
 8   Add_Lat_Lon             1451 non-null   object 
dtypes: float64(2), int64(2), object(5)
memory usage: 102.1+ KB


## Distance Measurement from the CPH Central Station

In [187]:
from geopy.distance import great_circle

In [238]:
great_circle(df_bolig['CPH_Lat_Lon'][0], df_bolig['Add_Lat_Lon'][0]).kilometers

5.349847518546733

In [239]:
dist=[]
for i in range(len(df_bolig)):
    dis=great_circle(df_bolig['CPH_Lat_Lon'][i], df_bolig['Add_Lat_Lon'][i]).kilometers
    dist.append(dis)

In [253]:
df_bolig.drop("Distance_from_Central_St", axis=1, inplace=True)

In [254]:
df_bolig.insert(6, "Distance_Central_St_km", dist)

In [255]:
df_bolig.head()

,Apartment,Rent,Address,URL,Rooms,Area(m²),Distance_Central_St_km,CPH_Lat_Lon,Add_Lat_Lon
0,2 rm. · Apartment · 60 m²,8.700,"København, Vigerslevvej",https://www.boligportal.dk/en/rental-apartment...,2,60,5.349848,"(55.67439305996968, 12.564873126715405)","(55.657086753224576, 12.485280377686337)"
1,3 rm. · Apartment · 102 m²,16.500,"København, Hans Hedtofts Gade",https://www.boligportal.dk/en/rental-apartment...,3,102,1.393140,"(55.67439305996968, 12.564873126715405)","(55.66296135575674, 12.573964013115852)"
2,2 rm. · Apartment · 60 m²,8.429,"København, Romsdalsgade",https://www.boligportal.dk/en/rental-apartment...,2,60,2.857594,"(55.67439305996968, 12.564873126715405)","(55.6595067245476, 12.602015356442074)"
3,3 rm. · Apartment · 88 m²,12.000,"København, Guldborgvej",https://www.boligportal.dk/en/rental-apartment...,3,88,2.701299,"(55.67439305996968, 12.564873126715405)","(55.68589177924863, 12.526917841605133)"
4,1 rm. · Room · 9 m²,3.500,"København, Charlotte Muncks Vej",https://www.boligportal.dk/en/rental-rooms/k%C...,1,9,4.555849,"(55.67439305996968, 12.564873126715405)","(55.71243131235872, 12.537862092305762)"


# 6 Rooms Dataframe Extracted

In [300]:
Room6=df_bolig.loc[df_bolig['Rooms'] == 6]

In [301]:
Room6.shape

(15, 9)

In [302]:
Room6

,Apartment,Rent,Address,URL,Rooms,Area(m²),Distance_Central_St_km,CPH_Lat_Lon,Add_Lat_Lon
30,6 rm. · House · 153 m²,22.000,"København, Sæbyholmsvej",https://www.boligportal.dk/en/rental-houses/k%...,6,153,4.603178,"(55.67439305996968, 12.564873126715405)","(55.67519799487328, 12.491473106706982)"
96,6 rm. · Apartment · 182 m²,28.800,"København, Gammel Kongevej",https://www.boligportal.dk/en/rental-apartment...,6,182,0.650951,"(55.67439305996968, 12.564873126715405)","(55.6736054583134, 12.554586012089635)"
102,6 rm. · Apartment · 198 m²,30.000,"København, Sortedam Dossering",https://www.boligportal.dk/en/rental-apartment...,6,198,1.449822,"(55.67439305996968, 12.564873126715405)","(55.68736816668332, 12.562594562752952)"
118,6 rm. · House · 167 m²,24.000,"København, Nakskovvej",https://www.boligportal.dk/en/rental-houses/k%...,6,167,4.867386,"(55.67439305996968, 12.564873126715405)","(55.66534622978468, 12.488931049590505)"
165,6 rm. · Apartment · 152 m²,24.000,"København, Paludan Müllers Vej",https://www.boligportal.dk/en/rental-apartment...,6,152,1.417539,"(55.67439305996968, 12.564873126715405)","(55.67301003539963, 12.542399539464558)"
369,6 rm. · Apartment · 151 m²,19.300,"København, Øresundsvej",https://www.boligportal.dk/en/rental-apartment...,6,151,4.379975,"(55.67439305996968, 12.564873126715405)","(55.66217752047773, 12.631272276163509)"
370,6 rm. · Apartment · 151 m²,19.500,"København, Lergravsvej",https://www.boligportal.dk/en/rental-apartment...,6,151,3.657458,"(55.67439305996968, 12.564873126715405)","(55.66308797876275, 12.619642149294114)"
467,6 rm. · Apartment · 193 m²,27.068,"København, Sortedam Dossering",https://www.boligportal.dk/en/rental-apartment...,6,193,1.449822,"(55.67439305996968, 12.564873126715405)","(55.68736816668332, 12.562594562752952)"
476,6 rm. · Apartment · 170 m²,25.000,"København, Nørre Søgade",https://www.boligportal.dk/en/rental-apartment...,6,170,0.991101,"(55.67439305996968, 12.564873126715405)","(55.68320205807227, 12.562463281792615)"
537,6 rm. · House · 231 m²,45.000,"København, Svanemøllevej",https://www.boligportal.dk/en/rental-houses/k%...,6,231,5.060145,"(55.67439305996968, 12.564873126715405)","(55.719527260218584, 12.575186502612288)"


## Folium Libaray to Display Copenhagen Map
    
    - 6 Rooms are highlighted on Copenhagen map with their rents

In [303]:
center = [55.67439305996968, 12.564873126715405]
map_Copenhagen = folium.Map(location=center, zoom_start=13)

for i,r in Room6.iterrows():
    folium.Marker(location=r['Add_Lat_Lon'],
                    popup = r['Rent'],
                    tooltip ='6 Rooms').add_to(map_Copenhagen)
#display map
map_Copenhagen